## 1. Import data and generate cleaned data

In [1]:
import numpy as np
# import modin.config as modin_cfg
# modin_cfg.Engine.put("dask")
import pandas as pd
import sqlite3
import itertools

In [ ]:
con = sqlite3.connect("./spotify.sqlite")
con.text_factory = lambda x: x.decode(encoding='utf-8', errors='ignore')
pd.set_option('display.max_columns', None)

In [ ]:
albums_df = pd.read_sql_query("SELECT * from albums", con)
print(albums_df.shape)
albums_df.head()

In [ ]:
artists_df = pd.read_sql_query("SELECT * from artists", con)
print(artists_df.shape)
artists_df.head()

In [ ]:
audiofeatures_df = pd.read_sql_query("SELECT * from audio_features", con)
audiofeatures_df.rename(columns={'id': 'audio_feature_id'}, inplace=True)
print(audiofeatures_df.shape)
audiofeatures_df.head()

In [ ]:
genres_df = pd.read_sql_query("SELECT * from genres", con)
print(genres_df.shape)
genres_df.head()

In [ ]:
rAlbumsArtists_df = pd.read_sql_query("SELECT * from r_albums_artists", con)
print(rAlbumsArtists_df.shape)
rAlbumsArtists_df.head()

In [ ]:
rAlbumsTracks_df = pd.read_sql_query("SELECT * from r_albums_tracks", con)
print(rAlbumsTracks_df.shape)
rAlbumsTracks_df.head()

In [ ]:
rArtistGenre_df = pd.read_sql_query("SELECT * from r_artist_genre", con)
rArtistGenre_df = rArtistGenre_df.groupby('artist_id')['genre_id'].apply(list).reset_index()
print(rArtistGenre_df.shape)
rArtistGenre_df.head()

In [ ]:
rTrackArtist_df = pd.read_sql_query("SELECT * from r_track_artist", con)
rTrackArtist_df['track_id'] = rTrackArtist_df['track_id'].astype('|S')
print(rTrackArtist_df.shape)
rTrackArtist_df.head()

In [ ]:
tracks_df = pd.read_sql_query("SELECT * from tracks", con)
print(tracks_df.shape)
tracks_df.head()

In [ ]:
merged_tracks = pd.merge(rTrackArtist_df, rArtistGenre_df, left_on='artist_id', right_on='artist_id')
merged_tracks = pd.merge(merged_tracks, artists_df, left_on='artist_id', right_on='id')
merged_tracks.drop(['id', 'popularity', 'followers'],axis=1,inplace=True)
merged_tracks.rename(columns={'name': 'artist_name'}, inplace=True)
merged_tracks.head()

In [ ]:
merged_tracks['track_id'] = merged_tracks['track_id'].str.decode("utf-8")
merged_tracks.head()

In [ ]:
grouped_tracks = merged_tracks.groupby('track_id').agg({
    'artist_id': list,
    'genre_id': lambda x: list(set(itertools.chain(*x))),
    'artist_name': list
}).reset_index()

grouped_tracks.head()

In [ ]:
merged = pd.merge(tracks_df, grouped_tracks, left_on='id', right_on='track_id')
merged.drop(['preview_url', 'track_id', 'is_playable'], axis=1, inplace=True)
merged.head()

In [ ]:
merged = pd.merge(merged, audiofeatures_df, left_on='audio_feature_id', right_on='audio_feature_id')
merged.head()

In [ ]:
merged.drop(['analysis_url', 'duration_y', 'disc_number', 'track_number'], axis=1, inplace=True)
merged.rename(columns={'duration_x': 'duration'}, inplace=True)
merged.head()

In [ ]:
albums_df['release_date'] = albums_df['release_date'].clip(lower=-6000000000000)
albums_df['release_datetime'] = pd.to_datetime(albums_df['release_date'], origin='unix', unit='ms')
albums_df['year'] = albums_df['release_datetime'].dt.year
albums_df.head()

In [ ]:
albums_df.drop(['name', 'album_group', 'album_type', 'release_date', 'release_date', 'popularity', 'release_datetime'], axis=1, inplace=True)
albums_df.rename(columns={'id': 'album_id'}, inplace=True)
albums_df.head()

In [ ]:
merged_year = pd.merge(rAlbumsTracks_df, albums_df, left_on='album_id', right_on='album_id')
merged_year.head()

In [ ]:
merged2 = pd.merge(merged, merged_year, left_on='id', right_on='track_id')
merged2.head()

In [ ]:
merged2.drop(['audio_feature_id','artist_id', 'explicit', 'album_id', 'track_id'], axis=1, inplace=True)
merged2.head()

In [ ]:
merged2.to_pickle('./spotify-data-cleaned.pkl')

In [ ]:
spotify_data = pd.read_pickle('./spotify-data-cleaned.pkl')
spotify_data.head()

In [ ]:
print(spotify_data.shape)

In [ ]:
# filter by popularity for size reasons
spotify_data2 = spotify_data.loc[spotify_data['popularity'] >= 20]
spotify_data2.head()

In [ ]:
print(spotify_data2.shape)

In [ ]:
spotify_data2 = spotify_data2.loc[spotify_data['year'] >= 1900]

In [ ]:
print(spotify_data2.shape)

In [ ]:
spotify_data2.to_pickle('./spotify-data-cleaned-less.pkl')

## Get Feature Set from data

In [2]:
import numpy as np
import pandas as pd
spotify_data2 = pd.read_pickle('./spotify-data-cleaned-less.pkl')

In [3]:
def custom_round(x, base=5):
    return int(base * round(float(x)/base))

In [4]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name, round_to = -1): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    col = df[column]
    if round_to > 0:
        col = col.apply(lambda x: custom_round(x, base=round_to))
    tf_df = pd.get_dummies(col)
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

#function to build entire feature set
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genre_id'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year', 5) * 0.9
    popularity_ohe = ohe_prep(df, 'popularity','pop', 5) * 0.9

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) # * 0.2

    print(genre_df.shape, floats_scaled.shape, popularity_ohe.shape, year_ohe.shape)
    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final

In [6]:
float_cols = spotify_data2.dtypes[spotify_data2.dtypes == 'float64'].index.values
spotify_data2[float_cols].describe()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,845485.000000,845485.000000,845485.000000,845485.000000,845485.000000,845485.000000,845485.000000,845485.000000,845485.000000
mean,0.346637,0.566739,0.597515,0.155438,0.207512,-9.267084,0.097115,120.154367,0.502124
std,0.332785,0.179915,0.254843,0.309646,0.187105,5.446598,0.136409,30.299293,0.267135
min,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000
25%,0.035700,0.450000,0.421000,0.000000,0.096400,-11.304000,0.035000,96.071999,0.283000
50%,0.234000,0.582000,0.631000,0.000050,0.128000,-7.845000,0.047300,119.678001,0.501000
75%,0.634000,0.701000,0.808000,0.045900,0.261000,-5.650000,0.089000,139.876007,0.722000
max,0.996000,0.991000,1.000000,1.000000,1.000000,4.638000,0.971000,248.018997,1.000000


In [7]:
feature_set = create_feature_set(spotify_data2, float_cols=float_cols)
feature_set.head()

(845485, 2659) (845485, 9) (845485, 17) (845485, 25)


,genre|150,genre|21st,genre|432hz,genre|48g,genre|8d,genre|a3,genre|aarhus,genre|abc,genre|aberdeen,genre|abstract,...,year|1980,year|1985,year|1990,year|1995,year|2000,year|2005,year|2010,year|2015,year|2020,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,2g8HN35AnVGIk7B8yMucww
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,4E5IFAXCob6QqZaJMTw5YN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,1gSt2UlC7mtRtJIc5zqKWn
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,67IvfvAMYQzJEeHopvwMMW
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,2I9foKseoFQh07p6sD2voE


In [8]:
feature_set.shape

(845485, 2711)

In [9]:
feature_set.to_pickle('./feature_set.pkl')

## Make predictions on Spotify playlists

In [ ]:
feature_set = pd.read_pickle('./feature_set.pkl')
spotify_data2 = pd.read_pickle('./spotify-data-cleaned-less.pkl')

In [10]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

#client id and secret for my application
client_id = ''
client_secret = ''
username = '31destplc6gv6h5zvqnyn6t46x7y'

In [11]:
scope = 'user-library-read'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
#gather playlist names
id_name = {}
for i in sp.user_playlists(username)['items']:
    id_name[i['name']] = i['uri'].split(':')[2]
    
print(id_name)

{'👴🏻💀': '5rokJjIreNMU0qT7peYhhZ', 'chill \U0001f9ca': '28ltExH2dT8s4Mgaw0bvC8', 'ye': '2CFRMOnXvI7lF8cqMvnNGG', '10s': '6Qv0cWT4hfRQkdrEyEn84H'}


In [13]:
def create_necessary_outputs(playlist_name, id_dic, df, weight_factor):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    most_recent_date = playlist.iloc[0,-1]
    
    for ix, row in playlist.iterrows():
        playlist.loc[ix,'months_from_recent'] = ((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist['weight'] = playlist['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    return playlist

In [14]:
playlist = create_necessary_outputs('chill \U0001f9ca', id_name, spotify_data2, 1.09)

In [15]:
playlist.head(100)

,artist,name,id,date_added,months_from_recent,weight
48,The Weeknd,Twenty Eight,1FJrsh1NbV9eRw5JQt3M4e,2023-06-09 01:12:22+00:00,0.000000,1.000000
46,Beach House,Space Song,7H0ya83CMmgFcOhw0UB6ow,2023-05-24 04:08:52+00:00,0.500000,0.957826
45,Daniel Caesar,Japanese Denim,1boXOL0ua7N2iCOUVI1p9F,2023-05-23 18:29:26+00:00,0.533333,0.955079
44,Mac Miller,Dang! (feat. Anderson .Paak),5iUQMwxUPdJBFeGkePtM66,2023-05-23 04:22:38+00:00,0.533333,0.955079
42,"Tyler, The Creator",RUNNING OUT OF TIME,5QvBXUm5MglLJ3iBfTX2Wo,2023-05-22 22:47:37+00:00,0.566667,0.952339
41,"Tyler, The Creator",A BOY IS A GUN*,1nXZnTALNXiPlvXotqHm66,2023-05-22 22:47:27+00:00,0.566667,0.952339
40,The Weeknd,After Hours,2p8IUWQDrpjuFltbdgLOag,2023-05-08 03:21:16+00:00,1.033333,0.914800
38,Frank Ocean,Pink Matter,1fOkmYW3ZFkkjIdOZSf596,2023-05-08 03:20:47+00:00,1.033333,0.914800
31,Kali Uchis,Melting,2kSb3wYSOV996xA2NSmpck,2023-03-31 21:56:40+00:00,2.300000,0.820199
29,Frank Ocean,Super Rich Kids,0725YWm6Z0TpZ6wrNk64Eb,2023-03-22 06:19:19+00:00,2.600000,0.799266


In [16]:
def generate_playlist_feature(feature_set, playlist_df):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    playlist_df.set_index('id', inplace=True)
    feature_set.set_index('id', inplace=True)
    in_playlist = feature_set.loc[feature_set.index.intersection(playlist_df.index)]
    playlist.reset_index(inplace=True)
    feature_set.reset_index(inplace=True)
    in_playlist.reset_index(inplace=True)
    
    feature_set_playlist = pd.merge(in_playlist, playlist_df, left_on = 'id', right_on='id')
    features_df = feature_set_playlist.drop(columns=['date_added','months_from_recent', 'id', 'artist', 'name'])

    # Step 2: Multiply each feature by the weight
    weighted_features = features_df.multiply(features_df['weight'], axis=0)
    # Step 3: Sum the weighted features
    weighted_sum = weighted_features.drop(columns=['weight'])
    weighted_sum = weighted_sum.sum(axis=0)
    
    return weighted_sum

In [17]:
playlist_features = generate_playlist_feature(feature_set, playlist)
print(playlist_features)

genre|150      0.000000
genre|21st     0.000000
genre|432hz    0.000000
genre|48g      0.000000
genre|8d       0.000000
                 ...   
year|2000      0.000000
year|2005      0.000000
year|2010      4.480188
year|2015      8.772569
year|2020      6.012584
Length: 2710, dtype: float64


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
def generate_playlist_recos(df, features, feature_set, playlist, n=10):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    df['sim'] = cosine_similarity(feature_set.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    top_sim = df.sort_values('sim',ascending = False).head(n + playlist.shape[0])
    filtered_top_sim = top_sim[~top_sim['id'].isin(playlist['id'])]
    
    return filtered_top_sim.head(n)

In [19]:
top_sim = generate_playlist_recos(spotify_data2, playlist_features, feature_set, playlist, 20)
top_sim.head(20)

,id,duration,name,popularity,genre_id,artist_name,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year,sim
98332,6Nle9hKrkL1wQpwNfEkxjh,210285,Chanel,82,"[hip hop, neo soul, alternative r&b, lgbtq+ hi...",[Frank Ocean],0.87400,0.776,0.503,0.000000,0,0.1120,-5.732,0,0.2370,110.134003,4,0.473,2017,0.870080
98735,5tz69p7tJuGPeMGwNTxYuV,250173,1-800-273-8255,80,"[hip hop, conscious hip hop, dmv rap, alternat...","[Alessia Cara, Logic, Khalid]",0.56900,0.620,0.574,0.000000,5,0.1900,-7.788,0,0.0479,100.023003,4,0.357,2017,0.851680
91807,6or1bKJiZ06IlK0vFvY75k,363521,Rap God,78,"[rap, hip hop, detroit hip hop]",[Eminem],0.39700,0.708,0.843,0.000000,7,0.7990,-2.660,1,0.3140,148.139999,4,0.625,2013,0.846514
7451713,0puf9yIluy9W0vpMEUoAnN,199386,Bang Bang,80,"[hip pop, post-teen pop, viral pop, pop rap, r...","[Nicki Minaj, Jessie J, Ariana Grande]",0.26000,0.706,0.786,0.000000,0,0.3800,-3.417,0,0.0910,150.028000,4,0.751,2014,0.846257
3853493,4X5f3vT8MRuXF68pfjNte5,233786,"F**kin' Problems (feat. Drake, 2 Chainz & Kend...",78,"[toronto rap, hip hop, conscious hip hop, sout...","[2 Chainz, Drake, Kendrick Lamar, A$AP Rocky]",0.02390,0.853,0.693,0.000000,1,0.1100,-6.870,1,0.2750,95.967003,4,0.662,2013,0.843458
99196,564oa00vY05d1uYnTEAAmE,183760,Herside Story,68,"[hip hop, dmv rap, alternative r&b, irish hip ...","[GoldLink, Hare Squead]",0.65600,0.885,0.595,0.000004,7,0.1050,-5.487,1,0.2860,148.033997,4,0.754,2017,0.842535
268750,7tr2za8SQg2CI8EDgrdtNl,230813,Slide (feat. Frank Ocean & Migos),70,"[electro house, uk dance, hip hop, neo soul, s...","[Calvin Harris, Frank Ocean, Migos]",0.49800,0.736,0.795,0.000001,1,0.2540,-3.299,0,0.0545,104.066002,4,0.511,2017,0.841812
219237,364dI1bYnvamSnBJ8JcNzN,212868,Intentions,80,"[hip hop, southern hip hop, post-teen pop, mel...","[Justin Bieber, Quavo]",0.31700,0.811,0.553,0.000000,9,0.1050,-6.644,1,0.0552,148.014008,4,0.860,2020,0.839473
217346,2q0VexHJirnUPnEOhr2DxK,277520,Biking,71,"[hip hop, neo soul, alternative r&b, east coas...","[JAY-Z, Frank Ocean, Tyler, The Creator]",0.68100,0.673,0.463,0.000005,2,0.0907,-7.247,1,0.1910,147.981003,4,0.268,2017,0.838649
2216409,2MJVc9F85EPNiSfD1KKhC9,213480,Los Mensajes del Whatsapp,68,"[latin hip hop, mexican hip hop]",[Cartel De Santa],0.70700,0.813,0.783,0.000000,1,0.2190,-5.347,1,0.2150,129.207993,4,0.927,2014,0.837710


In [20]:
playlist = create_necessary_outputs('👴🏻💀', id_name, spotify_data2, 1)
playlist.head(100)

,artist,name,id,date_added,months_from_recent,weight
32,Michael Bublé,Feeling Good,3I09LQbHS3NSU46Ly3tPpR,2023-07-10 07:06:36+00:00,0.000000,1.0
31,Engelbert Humperdinck,A Man Without Love,0oUBuOO4g9P4lREqfqR5nq,2023-07-10 07:03:05+00:00,0.000000,1.0
30,Engelbert Humperdinck,Can't Take My Eyes Off You,7oHymG8r14Iv9zn5hFTNax,2023-07-10 07:00:40+00:00,0.000000,1.0
29,Tears For Fears,Everybody Wants To Rule The World,4RvWPyQ5RL0ao9LPZeSouE,2023-07-05 21:47:09+00:00,0.133333,1.0
28,The Chordettes,Mr. Sandman,1T3tP6HvCgDF8ewpRqwM1e,2023-07-05 20:09:59+00:00,0.133333,1.0
27,The Jackson 5,I'll Be There,5RdhBLmB4DyFHLglRrfx63,2023-05-09 03:24:03+00:00,2.066667,1.0
25,The Beach Boys,Wouldn't It Be Nice,2Gy7qnDwt8Z3MNxqat4CsK,2023-04-24 06:39:38+00:00,2.566667,1.0
24,The Beach Boys,Good Vibrations - Remastered 2001,5t9KYe0Fhd5cW6UYT4qP8f,2023-04-24 06:39:15+00:00,2.566667,1.0
23,The Beach Boys,Surfin' U.S.A. - Remastered 2001,0wz1LjDb9ZNEYwOmDJ3Q4b,2023-04-23 05:24:00+00:00,2.600000,1.0
22,The Doors,Break on Through (To the Other Side),6ToM0uwxtPKo9CMpbPGYvM,2023-04-18 17:58:21+00:00,2.733333,1.0


In [21]:
playlist_features = generate_playlist_feature(feature_set, playlist)
top_sim = generate_playlist_recos(spotify_data2, playlist_features, feature_set, playlist, 20)
top_sim.head(20)

,id,duration,name,popularity,genre_id,artist_name,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year,sim
25960,6FT83pFXKhDlXDsNJFAHWz,161906,You Never Can Tell,72,"[blues rock, classic rock, folk rock, rock, ro...",[Chuck Berry],0.731,0.525,0.852,0.000051,7,0.2180,-9.519,1,0.2220,157.404007,4,0.884,1964,0.886040
24824,3VbGCXWRiouAq8VyMYN2MI,141693,Love Me Do - Remastered 2009,70,"[classic rock, british invasion, rock, beatles...",[The Beatles],0.386,0.520,0.829,0.000062,0,0.2270,-6.228,1,0.0806,147.996994,4,0.744,1963,0.864270
3870466,29SyMC0plk6qw8NMF7lfRL,134253,You Really Got Me - Mono Mix,73,"[glam rock, classic rock, folk rock, british i...",[The Kinks],0.493,0.573,0.939,0.000000,8,0.0994,-6.441,1,0.0808,137.382004,4,0.963,1964,0.855336
29511,3G7tRC24Uh09Hmp1KZ7LQ2,167373,I'm a Believer - 2006 Remaster,76,"[bubblegum pop, classic rock, folk rock, sunsh...",[The Monkees],0.707,0.526,0.775,0.000020,0,0.2170,-6.050,1,0.0352,80.106003,4,0.962,1967,0.853873
4517407,3qt3ybU6MScMxMKL2TYBvt,163000,No Particular Place To Go,64,"[blues rock, classic rock, folk rock, rock, ro...",[Chuck Berry],0.425,0.596,0.791,0.000295,7,0.0886,-8.880,1,0.0446,129.237000,4,0.980,1964,0.853213
932391,1udKn1oNKYQSQ9OmiIWCMu,125826,Do You Believe in Magic?,70,"[bubblegum pop, classic rock, folk rock, mello...",[The Lovin' Spoonful],0.213,0.553,0.738,0.000000,0,0.3390,-7.963,1,0.0457,138.602005,4,0.759,1965,0.852827
27064,3dzW0SoVNyJTWWJimX8stj,145293,She's Not There,65,"[bubblegum pop, classic rock, folk rock, briti...",[The Zombies],0.579,0.523,0.803,0.000000,4,0.0948,-5.189,0,0.0450,132.315994,4,0.898,1965,0.851555
24578,5ZBeML7Lf3FMEVviTyvi8l,155226,Twist And Shout - Remastered 2009,76,"[classic rock, british invasion, rock, beatles...",[The Beatles],0.641,0.482,0.849,0.000008,2,0.0414,-9.198,1,0.0452,124.630997,4,0.937,1963,0.851018
3918181,3KiexfmhxHvG5IgAElmTkd,173946,I Saw Her Standing There - Remastered 2009,70,"[classic rock, british invasion, rock, beatles...",[The Beatles],0.270,0.491,0.801,0.000000,4,0.0665,-9.835,1,0.0361,160.108994,4,0.971,1963,0.850280
25928,5UvE5QlINnvEc7aFO14DVL,109706,Great Balls Of Fire,69,"[piano rock, brill building pop, rock-and-roll...",[Jerry Lee Lewis],0.458,0.400,0.818,0.000003,0,0.1320,-11.238,1,0.0800,166.287003,4,0.936,1964,0.848985
